#### Connect Firebase

In [ ]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

import pandas as pd

# Use a service account
cred = credentials.Certificate('doctor_keys/doctor-4164c-firebase-adminsdk-2s6v3-37d33ca77e.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

#### Get a list of documents 

Data uploaded by gastroenterologists for every visited patient

In [ ]:
getTodo = db.collection(u'TodoDoc').stream()
td = [i.id for i in getTodo]

#### Batch wise save data for every 10 hours

In [ ]:
import schedule
import time

def job():
    for i in td:
    asd = db.collection(u'csv').document(i).get().to_dict()
    new_df = pd.DataFrame.from_dict(asd, orient="index").T
    new_df = new_df[["Wellbeing", "SiteDisease", "BodyWeight", "Appetite", "AvgTemp",
                "Above100F", "Vomit", "LiquidStools", "Diarrhea", "BowelMovements",
                "Arthritis", "Skin", "Eye", "AbdominalPain", "Pain", "PainAwake", "AbdominalMass",
                "AbdominalTenderness", "Fissure", "Blood", "Result"]]
    new_df[['BodyWeight', 'AvgTemp']] = new_df[['BodyWeight', 'AvgTemp']].astype('int32')
    old_data = pd.read_csv("data.csv")
    frames = [old_data, new_df]
    result = pd.concat(frames, axis=0, ignore_index = True)
    result.to_csv('data.csv', index=False)
    db.collection(u'TodoDoc').document(i).delete() # deletes document

schedule.every(10).hour.do(job)

while True:
    schedule.run_pending()
    time.sleep(1)